In [ ]:
# Ensure repo and src/ are importable and .env is loaded
import notebooks._bootstrap_src_path  # noqa: F401


In [ ]:
# Dependencies are managed via the repository's virtual environment.
# Please install once from the project root (e.g., using requirements.txt) rather than in-notebook.
# This cell intentionally avoids in-notebook `%pip install` commands.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from utils.markdown_utils import for_each_markdown_file
from tqdm import tqdm
import pandas as pd
from openai import OpenAI
import json

tqdm.pandas()

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env (bootstrap also does this; safe to call again)
load_dotenv()

# Expect OPENAI_API_KEY to be set in environment
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Generate Questions with Ground Truth and References

In [16]:
all_data = []
number_of_questions = 10

def generate_questions_for_md_file(filename, bytes):
    print("Generating QA for", filename)
    content = bytes.decode("utf-8")

    prompt = f'''
    Basierend auf dem folgenden Text:
    """
    {content}
    """

    Aufgabe:
    Formuliere {number_of_questions} realistische Patientenfragen, die sich **konkret auf diesen Text beziehen** und sich **ausschließlich mit Informationen aus diesem Text beantworten lassen**.
    Die Frage soll so formuliert sein, wie ein Patient sie in einer Klinik stellen würde.

    Für jede Frage gib bitte ein JSON-Objekt mit folgendem Aufbau zurück:

    - `question`: Eine realistisch formulierte Patientenfrage.
    - `references`: Eine Liste aller relevanten Textstellen (nur einzelne Sätze ohne Überschriften), die zur Beantwortung der Frage notwendig sind.  
      → Die Referenzen müssen **1:1 dem Originaltext entsprechen**, damit sie später exakt gematcht werden können.
    - `groundTruth`: Eine sachliche und vollständige Antwort auf die Frage – **nur basierend auf dem gegebenen Text**.

    Gib das Ergebnis als **valide JSON-Liste mit genau {number_of_questions} Objekten** zurück. Kein Freitext, keine Kommentare, keine Einleitung – nur das JSON-Array.

    Beispielstruktur:
    [
        {{
            "question": string,
            "references": string[],
            "groundTruth": string
        }},
        ...
    ]
    '''

    try:
        response = client.responses.create(
            input=prompt,
            model="gpt-4.1",
            temperature=1.0,
        )

        text = response.output_text.strip()

        if text.startswith("```json"):
            text = text.replace("```json", "").replace("```", "").strip()

        qa = json.loads(text)

        for entry in qa:
            entry["source_file"] = filename

        return qa

    except Exception as e:
        print(f"Error generating QA for file {filename}: {e}")
        return []

all_data = []

def wrapper(filename, bytes):
    all_data.extend(generate_questions_for_md_file(filename, bytes))

for_each_markdown_file("data/md_files", wrapper)

df = pd.DataFrame(all_data)

Processing Markdown files:   0%|          | 0/7 [00:00<?, ?it/s]

Generating QA for Äußere Wendung.md


Processing Markdown files:  14%|█▍        | 1/7 [00:49<04:57, 49.52s/it]

Generating QA for Kaiserschnitt.md


Processing Markdown files:  29%|██▊       | 2/7 [01:30<03:43, 44.61s/it]

...

Processing Markdown files:  71%|███████▏  | 5/7 [02:46<00:58, 29.12s/it]

Generating QA for Einleitung der Geburt.md


Processing Markdown files:  86%|████████▌ | 6/7 [03:24<00:31, 31.97s/it]

Generating QA for Äußere Wendung.md


Processing Markdown files: 100%|██████████| 7/7 [04:06<00:00, 35.15s/it]


In [ ]:
df["question"]

## Generate Question Variations

In [21]:
for index, row in tqdm(df.iterrows(), total=len(df), desc="Generating question variations"):
    original_question = row["question"]

    prompt = f"""
    Du erhältst eine medizinische Patientenfrage zur Geburtshilfe. Erzeuge daraus 9 umformulierte Varianten, wie sie typischer Weise von Patienten in einem Krankenhaus gestellt werden:

    1. **Kontextbezogen**: Erstelle eine Mini-Konversation (Patient → Assistent → Patient). 
        - Die letzte Patientenfrage **muss inhaltlich exakt der ursprünglichen Frage entsprechen** (die erste Frage führt nur darauf hin).
        - **Wichtig:** In der letzten Frage dürfen keinerlei Fachbegriffe, Schlüsselwörter oder eindeutige Begriffe aus der ursprünglichen Frage vorkommen. Stattdessen soll sie ausschließlich mit Platzhaltern, Pronomen oder allgemeinen Formulierungen gestellt werden, sodass sie ohne den vorherigen Chat völlig unverständlich ist (z. B. „Wieso geht das nicht?“ oder „Und was passiert dann genau?“).
    2. **Umgangssprachlich**: Formuliere die ursprüngliche Frage in lockerer, einfacher Alltagssprache, so wie ein Patient sie in einem Chat oder per Nachricht schreiben würde. Die Formulierung soll kurz, direkt und ungezwungen sein, gerne mit verkürzten Sätzen oder Umgangswörtern, aber ohne übertriebenen Slang oder Jugendsprache.
    3. **Englisch**: Die gleiche Frage aber auf Englisch.
    4. **Keyword**: Sehr kurz, ähnlich einer Google Suche.
    5. **Kurz**: Kurze Frage, nur wenige Worte lang.
    6. **Lang**: Lange Frage, die sich über mehrere Sätze zieht.
    7. **Fachlich**: Die Frage, aber gestellt von einem Patienten der sich gut auskennt und viele Fachwörter verwendet.
    8. **Fehler**: Die Frage, aber mit einem leichten Fehler drin, also z.B. die Operation nach der gefragt wird sehr falsch geschrieben.
    9. **Türkisch**: Die gleiche Frage, aber auf Türkisch.

    Ursprüngliche Frage:
    \"{original_question}\"
    
    Gib bitte ausschließlich ein **valide formatiertes JSON-Objekt** mit genau dieser Struktur zurück:

    {{
    "contextual": [
        {{
        "role": "user",
        "message": "<erste Frage des Patienten>"
        }},
        {{
        "role": "assistant",
        "message": "<Antwort oder Teilerklärung des Chatbots>"
        }},
        {{
        "role": "user",
        "message": "<Rückfrage, die sich nur mit dem obigen Kontext ergibt>"
        }}
    ],
    "slang": "<Frage in umgangssprachlichem Deutsch>",
    "english": "<Frage auf Englisch>",
    "keyword": "<Frage als kurze Keyword-Suche>",
    "short": "<kurze Frage, nur wenige Worte>",
    "long": "<lange Frage, mehrere Sätze>",
    "technical": "<fachlich formulierte Frage mit Fachbegriffen>",
    "mistake": "<Frage mit leichtem Schreib- oder Inhaltsfehler>",
    "turkish": "<Frage auf Türkisch>"
    }}

    Keine weiteren Ausgaben oder Kommentare.
    """

    try:
        response = client.responses.create(
            input=prompt,
            model="gpt-4.1-mini",
            temperature=1.0,
        )

        text = response.output_text.strip()

        if text.startswith("```json"):
            text = text.replace("```json", "").replace("```", "").strip()

        variations = json.loads(text)

        df.at[index, "variations"] = json.dumps(variations, ensure_ascii=False)

    except Exception as e:
        print(f"⚠️ Fehler bei Zeile {index} (Frage: '{original_question}'): {e}")
        df.at[index, "variations"] = None

Generating question variations: 100%|██████████| 61/61 [06:41<00:00,  6.59s/it]


In [22]:
df.to_pickle("data/datasets/question_answers_dataset_raw.pkl")

## Validation

In [3]:
df = pd.read_pickle("data/datasets/question_answers_dataset_raw.pkl")

In [23]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import json

# create a new column with pretty-printed JSON
df["variations_pretty"] = df["variations"].apply(
    lambda x: json.dumps(x, indent=4)
)

# display only the pretty column
from IPython.display import display, HTML

# show as HTML with <pre> tags for formatting
display(HTML(df[["variations_pretty"]].to_html(escape=False)))

,variations_pretty
0,"""{\""contextual\"": [{\""role\"": \""user\"", \""message\"": \""Ich habe von der \u00e4u\u00dferen Wendung geh\u00f6rt, k\u00f6nnen Sie mir sagen, was genau dabei gemacht wird?\""}, {\""role\"": \""assistant\"", \""message\"": \""Gerne! Dabei wird versucht, das Baby von einer Becken- in eine Kopflage zu drehen, um eine komplikationslosere Geburt zu erm\u00f6glichen. Vorher wird dein Zustand genau kontrolliert, um sicherzugehen, dass die Drehung sicher durchgef\u00fchrt werden kann.\""}, {\""role\"": \""user\"", \""message\"": \""Und was passiert dann genau davor?\""}], \""slang\"": \""Wie genau l\u00e4uft so ne \u00e4u\u00dfere Wendung ab und was macht man davor?\"", \""english\"": \""How is an external version performed exactly and what happens beforehand?\"", \""keyword\"": \""\u00e4u\u00dfere Wendung Ablauf Vorbereitung\"", \""short\"": \""Wie l\u00e4uft eine \u00e4u\u00dfere Wendung ab?\"", \""long\"": \""K\u00f6nnen Sie mir bitte ausf\u00fchrlich erkl\u00e4ren, wie eine \u00e4u\u00dfere Wendung durchgef\u00fchrt wird? Mich interessiert auch, welche Vorbereitungsschritte davor notwendig sind und was genau in diesem Prozess passiert.\"", \""technical\"": \""K\u00f6nnten Sie detailliert erl\u00e4utern, wie eine externe Cephalic Version (\u00e4u\u00dfere Wendung) abl\u00e4uft und welche pr\u00e4partalen Ma\u00dfnahmen zur Sicherstellung der fetalen und maternalen Sicherheit vor dem Eingriff getroffen werden?\"", \""mistake\"": \""Wie l\u00e4uft eine \u00e4ussere Wendung genau ab und was passiert davor?\"", \""turkish\"": \""D\u0131\u015f ters d\u00f6nd\u00fcrme (d\u0131\u015f hamle) nas\u0131l yap\u0131l\u0131r ve \u00f6ncesinde neler olur?\""}"""
1,"""{\""contextual\"": [{\""role\"": \""user\"", \""message\"": \""Ich habe geh\u00f6rt, es gibt eine M\u00f6glichkeit, das Baby in der Geb\u00e4rmutter zu drehen. K\u00f6nnen Sie mir sagen, wann man das am besten macht?\""}, {\""role\"": \""assistant\"", \""message\"": \""Ja, das nennt sich \u00e4u\u00dfere Wendung und wird normalerweise ab der 36. Schwangerschaftswoche durchgef\u00fchrt, um die Lage des Babys zu \u00e4ndern.\""}, {\""role\"": \""user\"", \""message\"": \""Und wann genau ist da der beste Moment?\""}], \""slang\"": \""Wann kann man das Baby am besten drehen lassen?\"", \""english\"": \""When is the best time for an external cephalic version?\"", \""keyword\"": \""best time external version baby\"", \""short\"": \""Wann beste Wendung?\"", \""long\"": \""Ich w\u00fcrde gerne wissen, wann genau der optimale Zeitpunkt ist, um eine \u00e4u\u00dfere Wendung bei meinem Baby durchzuf\u00fchren. K\u00f6nnen Sie mir erkl\u00e4ren, ab welcher Schwangerschaftswoche das sinnvoll ist und warum?\"", \""technical\"": \""Ab welchem Gestationsalter empfiehlt sich eine \u00e4u\u00dfere Wendung zur Korrektur einer Beckenendlage unter Ber\u00fccksichtigung der fetalen Reife und maternalen Faktoren?\"", \""mistake\"": \""Wann ist der beste Zeitpunkt f\u00fcr eine \u00e4u\u00dfere Wenung?\"", \""turkish\"": \""D\u0131\u015f d\u00f6nd\u00fcrme i\u00e7in en uygun zaman ne zamand\u0131r?\""}"""
2,"""{\""contextual\"": [{\""role\"": \""user\"", \""message\"": \""Ich habe geh\u00f6rt, man kann manchmal das Baby vor der Geburt noch drehen. Ist das sicher?\""}, {\""role\"": \""assistant\"", \""message\"": \""Die \u00e4u\u00dfere Wendung ist eine Methode, um das Baby aus einer ung\u00fcnstigen Lage zu drehen. Es gibt einige Risiken und m\u00f6gliche Komplikationen, die dabei auftreten k\u00f6nnen.\""}, {\""role\"": \""user\"", \""message\"": \""Und was kann da genau schiefgehen?\""}], \""slang\"": \""Was kann denn alles schiefgehen, wenn man das Baby von au\u00dfen drehen will?\"", \""english\"": \""What risks and complications can occur during an external version?\"", \""keyword\"": \""Risiken \u00e4u\u00dfere Wendung Komplikationen\"", \""short\"": \""Welche Risiken bei \u00e4u\u00dferer Wendung?\"", \""long\"": \""Ich w\u00fcrde gerne wissen, welche m\u00f6glichen gesun

In [1]:
import pandas as pd

df = pd.read_pickle("data/qa_with_docs_flat/question_answers_docs_word_100_20_dataset_flat.pkl")

In [7]:
import pandas as pd

# Adjust pandas display options to show full text in columns
pd.set_option('display.max_colwidth', None)

# Now print the full sentences
print(df[df["variant"] == "contextual"]["question"].head(60))

1                                                                                                Und was passiert dann genau davor?
11                                                                                          Und wann genau ist da der beste Moment?
21                                                                                               Und was kann da genau schiefgehen?
31                                                            Kann man mir sagen, welche Möglichkeiten es gibt, das Kind zu drehen?
41                                                                      Und wann genau kann man da so eine Sache nicht durchführen?
51                                                         Also, wie groß ist denn eigentlich die Chance, dass das wirklich klappt?
61                                                                                     Also, tut das weh und brauche ich Betäubung?
71                                                                          

In [8]:
len(df)

610